In [16]:
import pandas as pd
import requests
import time
import datetime
from tqdm import tqdm
from ta import add_all_ta_features
import numpy as np

In [2]:
#Initial Configuration
symbol = 'BTCUSDT'
interval = '1m'
start_date = '2016-01-01'
end_date = '2024-07-11'

In [3]:

# Función para obtener datos históricos de Binance API
def get_binance_bars(symbol, interval, start_time, end_time=None):
    base_url = 'https://api.binance.com'
    endpoint = '/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Límite máximo de datos por solicitud
    }
    
    response = requests.get(base_url + endpoint, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                     'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [5]:

# Convertir fechas a milisegundos
start_time = int(time.mktime(datetime.datetime.strptime(start_date, "%Y-%m-%d").timetuple()) * 1000)
end_time = int(time.mktime(datetime.datetime.strptime(end_date, "%Y-%m-%d").timetuple()) * 1000)

# Calcular el número total de minutos para la barra de progreso
total_minutes = (end_time - start_time) // 60000

# Descargar datos
df_list = []
current_time = start_time

with tqdm(total=total_minutes) as pbar:
    while current_time < end_time:
        df = get_binance_bars(symbol, interval, current_time, end_time)
        if df.empty:
            break
        df_list.append(df)
        current_time = int(df['timestamp'].iloc[-1].timestamp() * 1000) + 60000  # Avanzar 1 minuto más allá del último timestamp
        pbar.update(len(df))  # Actualizar la barra de progreso
        # Esperar para evitar límites de tasa
        time.sleep(0.5)

# Concatenar todos los datos descargados
df_all = pd.concat(df_list, ignore_index=True)

# Guardar en un archivo CSV
df_all.to_csv('bitcoin_minute_data.csv', index=False)

print("Datos descargados y guardados en 'bitcoin_1m_data.csv'")

 81%|████████  | 3619809/4484100 [1:05:30<15:38, 920.94it/s] 


Datos descargados y guardados en 'bitcoin_minute_data.csv'


In [19]:
# Leer el archivo CSV
df_all = pd.read_csv('../data/bitcoin_1min_data.csv')

# Cálculo de datos agregados para timeframes superiores
df_all['timestamp'] = pd.to_datetime(df_all['timestamp'])
df_all.set_index('timestamp', inplace=True)

# Calcular indicadores técnicos
df_all = add_all_ta_features(df_all, open="open", high="high", low="low", close="close", volume="volume")

# Calcular log returns
df_all['log_return'] = np.log(df_all['close'].astype(float) / df_all['close'].astype(float).shift(1))

# Guardar los datos agregados y las características técnicas en un archivo CSV
df_all.to_csv(f'../data/bitcoin_{timeframe}_data_ta.csv')

/opt/homebrew/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


In [18]:


# Crear un DataFrame para cada timeframe superior
timeframes = ['5min', '15min', '1h', '4h', '1d']
aggregated_dfs = {}

for timeframe in timeframes:
    df_agg = df_all.resample(timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'quote_asset_volume': 'sum',
        'number_of_trades': 'sum',
        'taker_buy_base_asset_volume': 'sum',
        'taker_buy_quote_asset_volume': 'sum'
    }).dropna()

    # Calcular indicadores técnicos
    df_agg = add_all_ta_features(df_agg, open="open", high="high", low="low", close="close", volume="volume")

    # Calcular log returns
    df_agg['log_return'] = np.log(df_agg['close'].astype(float) / df_agg['close'].astype(float).shift(1))

    # Guardar los datos agregados y las características técnicas en un archivo CSV
    df_agg.to_csv(f'../data/bitcoin_{timeframe}_data.csv')

print("Datos descargados y guardados en archivos CSV para cada timeframe")

/opt/homebrew/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/homebrew/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/homebrew/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/homebrew/lib/python3.10/site-

Datos descargados y guardados en archivos CSV para cada timeframe
